<center><h2>Sector Data Package Demo<h2></center>

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show/hide the raw code."></form>''')

In [2]:
# import alphalens
import sys
import os
import re
# import datetime
# import getpass
# import logging
# import sklearn
# from sklearn.base import clone
import ipywidgets as widgets
# import pandas as pd
from datetime import datetime
# import statsmodels.api as sm_api
# from sklearn.preprocessing import StandardScaler

%matplotlib inline
%config IPCompleter.greedy=True
%autosave 0

import pandas as pd
import numpy as np
import plotly.graph_objs as go
# from dslab.dsdata import MLData
# from dslab.dsmodel import MLModel
from ipywidgets import *
from IPython.display import display, clear_output, Image, Javascript, SVG
from plotly.offline import init_notebook_mode, iplot
from plotly.tools import FigureFactory as FF       

# Initialize data and model object instances
# dsData = MLData(regret=True)
# model_obj = MLModel()

# for plotly
init_notebook_mode(connected=True)

Autosave disabled


In [3]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

In [4]:
# import functools
# import scipy.interpolate
# import itertools
# import math
# from scipy.optimize import minimize
# from scipy.signal import fftconvolve
# from scipy.stats import norm
# import copy

In [5]:
# display settings

# widgets
row_layout = Layout(display='flex', flex_flow='row', align_items='center')
col_layout = dict(display='flex', flex_flow='column', justify_content='space-between', width='auto')

def leftm(m=10):
    '''Set left margin'''
    return '0px 0px 0px {}px'.format(m)

max_col = IntSlider(value=50, max=100, width='600px')
max_row = IntSlider(value=100, max=250, width='600px')

bs_row1 = Box([Label('Max Columns to Display:', layout=Layout(width='200px')),  max_col], layout=row_layout)
bs_row2 = Box([Label('Max Rows to Display:', layout=Layout(width='200px')),  max_row], layout=row_layout)
# display(Box([bs_row1, bs_row2], layout=Layout(**col_layout)))

# function
pd.options.display.max_columns = max_col.value
pd.options.display.max_rows = max_row.value

def chg_setting(chg):
    '''Change basic settings.'''
    pd.options.display.max_rows = max_row.value
    pd.options.display.max_columns = max_col.value

max_col.observe(chg_setting, names='value')
max_row.observe(chg_setting, names='value')

## 1. Load Data

In [6]:
# load data
# widgets
filepath = Text(description='File Path:', value='fake_data.csv')
loaddata_button = Button(description="Load Data", button_style='success', margin=leftm(650))
prog = FloatProgress(value=0, min=0, max=10, step=1, description='Progress:',
                     margin=leftm(550), width='250px')
display(filepath, loaddata_button, prog)

# functions
def loaddata(b):
    '''Load data.'''
#     clear_output()
#     display(filepath, loaddata_button, prog)
    global all_tickers, all_factors
    prog.value = 0
    prog.description = 'Start...'
    filepath_ = filepath.value # os.path.join(PATH.value, filename.value)
    dsData.read_data(filepath_)  # read data
    prog.value = 5
    prog.description = 'In Progress...'
    dsData.save_current()  # save a copy of current snapshot
    prog.value = 10
    prog.description = 'Done!'
    dsData.infer_categorical()
    clear_output()
    print('\n', 'Data Head')
    display(dsData.head())  # display data head
    print('\n', 'Data Tail')
    display(dsData.tail())  # display data tail
    print('\n', 'Summary Statistics')    
    display(dsData.get_summary())  # display summary statistics
    
    all_tickers = list(dsData.data['symbol'].unique())
    all_factors = [j for j in dsData.data.columns if j not in ['date', 'sid', 'symbol', 'Sector']]

loaddata_button.on_click(loaddata)

Text(value='fake_data.csv', description='File Path:')

Button(button_style='success', description='Load Data', style=ButtonStyle())

FloatProgress(value=0.0, description='Progress:', max=10.0)

## 2. Explore Data

In [7]:
# Select Graph

# widgets
chart_select = ToggleButtons(
    options=['Coverage', 'Pie Chart', 'Histogram', 'Scatter Plot', 'Correlations'],
    tooltips=['Explore feature coverage', 'Visualize single categorical variable', 'Visualize single numerical variable',
             'Visualize pairwise distributions among a group of numerical variables',
             'Visualize pairwise correlations among a group of numerical variables'],
    value='Correlations', 
    margin=leftm()
)

display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px')))

# functions
grph = HTML('')

def switch_chart(chg):
    '''Switch between graph type.'''
    global grph
    clear_output()
    display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px')))
    grph.close()
    grph = graph_mapping[chart_select.value]()
    display(grph)

chart_select.observe(switch_chart, names='value')

In [8]:
# Coverage check
# widgets
def make_coverage():    
    coverage_b = Button(description="Coverage", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    
    row0 = Box([HTML('<h4>Pie Chart - Visualize Distributions of Categorical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
    row1 = Box([Label(layout=Layout(width='200px')), coverage_b], layout=row_layout)
    coverage_chart = Box([row0, row1], layout=Layout(**col_layout),
                    height='160px')    
    return(coverage_chart)

In [9]:
# Pie chart
# widgets
def make_pie():
    '''create widgets for pie chart'''
    global pie_filter, pie_select, pie_limit, group_less_freq, pieb
    pie_filter = Text(value='', width='500px')
    pie_select = Dropdown(options=dsData.get_header(sort=True), height='30px', width='200px',
                          margin=leftm())
    pie_limit = IntText(value=20, width='70px')
    group_less_freq = Checkbox(value=False)
    pieb = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    
    row0 = Box([HTML('<h4>Pie Chart - Visualize Distributions of Categorical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
#     row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), pie_filter, pieb], layout=row_layout)
    row1 = Box([Label('Pie Chart Plot: ', layout=Layout(width='200px')), pieb], layout=row_layout)
#     row1 = Box([pieb], layout=row_layout)
    row2 = Box([Label('Select a Categorical Column: ', layout=Layout(width='200px')), pie_select,
                Label('Limit on # of Values: ', layout=Layout(width='200px'), margin=leftm(20)), pie_limit], layout=row_layout)
    row3 = Box([Label('Group Everything Below the Limit as "others": ', layout=Layout(width='400px')), group_less_freq],
                layout=row_layout)
    pie_chart = Box([row0, row1, row2, row3], layout=Layout(**col_layout),
                    height='160px')

    pie_select.observe(plot_pie, names='value')
    pieb.on_click(plot_pie)
    return pie_chart


# functions
def plot_pie(chg):
    '''plot the pie chart'''
#     with warnings.catch_warnings():
#         print("ignore warning")
#         warnings.simplefilter('ignore')
    clear_output()
    display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px'))) ## kai test

    if pie_filter.value.strip() != '':
        try:
            vc = dsData.query_chain(pie_filter.value).ct_freq(pie_select.value, group_less_freq.value,
                                                           pie_limit.value)
            vc.astype('category').cat.remove_unused_categories(True)
        except Exception as e:
            print(e)
            vc = dsData.ct_freq(pie_select.value, group_less_freq.value, pie_limit.value)
    else:
        vc = dsData.ct_freq(pie_select.value, group_less_freq.value, pie_limit.value)
    trace = go.Pie(labels=list(vc.index), values=vc.values)
    figp = go.Figure(data=go.Data([trace]),
                     layout=dict(title='Break Down of {}'.format(pie_select.value.upper())))
    iplot(figp)

In [10]:
# Histogram

# widgets
def make_hist():
    '''make widgets for histogram'''
    global hist_filter, histb, hist_select, hist_min, hist_max, hist_bins
    hist_filter = Text(value='', width='500px')
    histb = Button(description="Plot", button_style='success', width='70px', height='30px',
                   margin=leftm(20))
    hist_select = Dropdown(options=dsData.get_header(['int', 'float'], sort=True), height='30px',
                           width='200px', margin=leftm())
    hist_min = FloatText(value=float('-Inf'), width='70px')
    hist_max = FloatText(value=float('Inf'), width='70px')
    hist_bins = IntText(value=10, width='70px')
    
    row0 = Box([HTML('<h4>Histogram - Visualize Distributions of Numerical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
#     row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), hist_filter, histb], layout=row_layout)
    row1 = Box([Label('Histogram Plot: ', layout=Layout(width='200px')), histb], layout=row_layout)
        
    row2 = Box([Label('Select a Numerical Column: ', layout=Layout(width='200px')), hist_select], layout=row_layout)
    row3 = Box([Label('Min: ', layout=Layout(width='200px')), hist_min, Label('Max: ', margin=leftm(20)), hist_max,
                Label('# of Bins: ', margin=leftm(20)), hist_bins], layout=row_layout)

    hist_chart = Box([row0, row1, row2, row3], layout=Layout(**col_layout), height='160px')
    
    hist_select.observe(plot_hist_new_col)
    histb.on_click(plot_hist)
    return hist_chart


# functions
def plot_hist_new_col(chg):
    '''plot histogram when switching columns'''
    hist_min.value = float('-Inf')
    hist_max.value = float('Inf')
    plot_hist(chg)
    
def plot_hist(chg):
    '''plot histogram'''
    clear_output()
    display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px'))) ## kai test
    if hist_filter.value.strip() != '':
        x = dsData.query(hist_filter.value)[hist_select.value]
    else:
        x = dsData.data[hist_select.value]
    n = len(x)
    x = x[x > hist_min.value]
    x = x[x < hist_max.value]
    print('Percentage covered (after filtering): {:.2f}%'.format(len(x) / n * 100))
    if hist_min.value == float('-Inf'):
        hist_min.value = np.round(np.min(x), 2) - 1
    if hist_max.value == float('Inf'):
        hist_max.value = np.round(np.max(x), 2) + 1

    size = (hist_max.value - hist_min.value) / hist_bins.value
    trh = go.Histogram(x=x, histnorm='percent', marker=dict(color='rgb(0,0,100)'),
                       xbins=dict(start=hist_min.value - 0.5, size=size, end=hist_max.value + 0.5))

    layout = dict(bargap= 0.015, hovermode= 'x',
                  title='Histogram for {}'.format(hist_select.value.upper()),
                  yaxis= dict(title='Percentage (%)', autorange= True, showticklabels= True))
    figh = go.Figure(data=go.Data([trh]), layout=layout)
    iplot(figh)

In [11]:
# Scatter Plot
# widgets
def make_scatter():
    '''Make widgets for scatter plot.'''
    global sp_filter, spb, sp_height, sp_select, sp_ccol
    num_cols = dsData.get_header(['int', 'float'], sort=True)
    sp_filter = Text(value='', width='500px')
    spb = Button(description="Plot", button_style='success', width='70px', height='30px',
                 margin=leftm(20))
    sp_height = np.min([300, np.max([len(num_cols) * 15, 100])])
    sp_select = SelectMultiple(options=num_cols, margin=leftm(),
                               height='{}px'.format(sp_height))
    sp_ccol = Dropdown(options=[None] + dsData.get_header('category', sort=True), height='30px', width='200px', margin=leftm())

    row0 = Box([HTML('<h4>Scatter Plot - Visualize Pairwise Distributions among a Group of Numerical '
                     'Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
#     row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), sp_filter, spb], layout=row_layout)
    row1 = Box([Label('Scatter Plot: ', layout=Layout(width='200px')), spb], layout=row_layout)
    row2 = Box([Label("Select Columns: ", layout=Layout(width='200px')), sp_select, Label("Group by: ", margin=leftm(30)), sp_ccol],
                layout=row_layout)
    scatter_chart = Box([row0, row1, row2], layout=Layout(**col_layout),
                        height='{}px'.format(sp_height + 110))
    spb.on_click(plot_scatter)
    return scatter_chart

# functions
def plot_scatter(chg):
    '''Generate scatter plot.'''
    clear_output()
    display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px'))) ## kai test

    if sp_filter.value.strip() != '':
        temp_data = dsData.query(sp_filter.value).copy(True)
    else:
        temp_data = dsData.data.copy(True)
    
    sel_cols = list(sp_select.value)
    if sp_ccol.value is not None:
        temp_data[sp_ccol.value].cat.remove_unused_categories(True)
        temp_data[sp_ccol.value] = temp_data[sp_ccol.value].astype('object')
        sel_cols.append(sp_ccol.value)

    temp = temp_data[sel_cols]
    
    fig = FF.create_scatterplotmatrix(temp, index=sp_ccol.value, diag='histogram', height=800,
                                      width=800)
    iplot(fig)

In [12]:
# Correlations
# widgets
def make_cor():
    '''Make widgets for correlations plot.'''
    global cor_filter, corb, cor_height, cor_select
    num_cols = dsData.get_header(['int', 'float'], sort=True)
    cor_filter = Text(value='', width='500px')
    corb = Button(description="Plot", button_style='success', width='70px', height='30px',
                  margin=leftm(20))
    cor_height = np.min([300, np.max([len(num_cols) * 15, 100])])
    cor_select = SelectMultiple(options=num_cols, margin=leftm(),
                                height='{}px'.format(cor_height))
    row0 = Box([HTML('<h4>Correlations - Visualize Correlations among a Group of Numerical Variables</h4>')],
                layout=Layout(display='flex', flex_flow='row', align_items='center',
                              justify_content='center', width='90%'))
#     row1 = Box([Label('Filter by Criterion: ', layout=Layout(width='200px')), cor_filter, corb], layout=row_layout)
    row1 = Box([Label('Correlation Plot: ', layout=Layout(width='200px')), corb], layout=row_layout)
    row2 = Box([Label("Select Columns: ", layout=Layout(width='200px')), cor_select], layout=row_layout)
    cor_chart = Box([row0, row1, row2], layout=Layout(**col_layout),
                    height='{}px'.format(cor_height + 100))
    corb.on_click(plot_cor)
    return cor_chart

# functions
def plot_cor(chg):
    '''Generate correlations plot.'''
    clear_output()
    display(HBox([Label('Graph Type: '), chart_select], layout=Layout(margin='0px 0px 30px 0px'))) ## kai test
    
    if cor_filter.value.strip() != '':
        temp_data = dsData.query(cor_filter.value).copy(True)
    else:
        temp_data = dsData.data.copy(True)
    
    sel_cols = list(cor_select.value)
    x, y = sel_cols, list(reversed(sel_cols))

    corr = temp_data[sel_cols].corr().round(2)
    z = np.flipud(corr.values)
    annotations = []
    for n, row in enumerate(z):
        for m, val in enumerate(row):
            var = z[n][m]
            annotations.append(
                dict(
                    text=str(val),
                    x=x[m], y=y[n],
                    xref='x1', yref='y1',
                    font=dict(color='black'),
                    showarrow=False)
                )

    colorscale = [[0, 'rgba(255,0,0,0.3)'], [0.5, 'rgba(255,255,0,0.3)'], [1, 'rgba(0,255,0,0.3)']]
    trace = go.Heatmap(x=x, y=y, z=z, zmin=-1, zmax=1, colorscale=colorscale, showscale=True)

    fig = go.Figure(data=[trace])
    fig['layout'].update(
        title="Correlations",
        annotations=annotations,
        xaxis=dict(ticks=''),
        yaxis=dict(ticks='', ticksuffix='  '),
        width=700,
        height=700,
        autosize=False
    )
    
    iplot(fig)

In [13]:
# graph_mapping = {'Pie Chart': make_pie, 'Histogram': make_hist, 'Box Plot': make_boxp,
#                  'Cross Tab': make_xtab, 'Scatter Plot': make_scatter, 'Correlations': make_cor}

graph_mapping = {'Coverage': make_coverage, 'Pie Chart': make_pie, 'Histogram': make_hist, 
                 'Scatter Plot': make_scatter, 'Correlations': make_cor}

## 4. Single Factor Evaluate

In [17]:
def cal_hit(ts):
    ts = ts.dropna()
    same_sign_counts, diff_sign_counts = sum(ts > 0), sum(ts <= 0)
    hit_rate = same_sign_counts / (same_sign_counts + diff_sign_counts)   
    return(hit_rate)

In [18]:
def run_single_factor_single_ticker(data):
    current_ticker = data['symbol'].unique()[0]
    print("single ticker: ", current_ticker)
    display(data.tail(5))
    compare_df = data[[factor_select.value, target_select.value]]
    print("Correlation:")
    display(compare_df.corr())
    print("Hit Rate (Same Sign):")
    check_sign = compare_df[factor_select.value] * compare_df[target_select.value]
    check_sign.index = data["date"]
    check_sign = check_sign.dropna()
    dates_remain = check_sign.index
    check_sign = pd.Series(np.sign(check_sign))
    check_sign.index = dates_remain
    display(check_sign.tail(5))
#     check_sign_mvg60 = check_sign.rolling(60).mean()
    check_sign_mvg60 = check_sign.rolling(60).apply(cal_hit)
    check_sign_mvg60.plot(rot=45)
    same_sign_counts, diff_sign_counts = sum(check_sign > 0), sum(check_sign <= 0)
    try:
        hit_rate = same_sign_counts / (same_sign_counts + diff_sign_counts)
    except:
        print("denominator zeros: ", same_sign_counts, diff_sign_counts)
        hit_rate = np.nan
    print("Over the history Hit Rate: ", hit_rate)
    

In [19]:
def run_single_factor(b):
    target_cols = [target_select.value]
#     cols = ['date', 'sid',  factor_select.value] + target_cols
    cols = ['date', 'sid', 'symbol', factor_select.value] + target_cols

    al_data = dsData.data[cols].copy()
    ticker_selected = ticker_select.value
    if ticker_selected != 'cross_tickers':
        al_data = al_data[al_data['symbol']==ticker_selected]
        run_single_factor_single_ticker(al_data)
        return()
        
    al_data = al_data.drop("symbol", axis=1)
    al_data = al_data.set_index(['date', 'sid'])

    factor_to_evaluate = al_data[factor_select.value]
    forward_returns = al_data[target_select.value]   

    data_for_alphalens = alphalens.utils.get_clean_factor(factor_to_evaluate, pd.DataFrame(forward_returns), max_loss=0.9) 

    data_for_alphalens = data_for_alphalens.reset_index()
    data_for_alphalens['date'] = pd.to_datetime(data_for_alphalens['date'])  ### make sure the dtype of date column
    data_for_alphalens.columns = ['date', 'asset'] + list(data_for_alphalens.columns[2:])
    data_for_alphalens = data_for_alphalens.set_index(['date', 'asset'])
    data_for_alphalens.columns = ['21D'] + list(data_for_alphalens.columns[1:])  ### alphalens only take standard column names
    # data_for_alphalens.columns = forward_returns_cols + list(data_for_alphalens.columns[1:])  ### alphalens only take standard column names

    alphalens.tears.create_returns_tear_sheet(data_for_alphalens)

    alphalens.tears.create_information_tear_sheet(
        factor_data=data_for_alphalens, group_neutral=False, by_group=False, set_context=False
    )

    alphalens.plotting.plot_quantile_statistics_table(data_for_alphalens)

#     alphalens.tears.create_full_tear_sheet(data_for_alphalens)

In [20]:
def alpha_single(b):
    '''Make widgets for single model.'''
    clear_output()
    display(individual_eval_button)
    global factor_select, target_select, ticker_select, run_eval_button
    factor_select = Dropdown(options=all_factors, height='30px', margin=leftm(), width='200px')
    target_select = Dropdown(options=all_factors, height='30px', margin=leftm(), width='200px')
    tickers_options = ['cross_tickers'] + all_tickers
    ticker_select = Dropdown(options=tickers_options, height='30px', margin=leftm(), width='200px')
    
    rows = []
    rows.append(Box([Label('Independent Factor: ', layout=Layout(width='200px')), factor_select], layout=row_layout))
    rows.append(Box([Label('Target Variable: ', layout=Layout(width='200px')), target_select], layout=row_layout))
    rows.append(Box([Label('Ticker Only: ', layout=Layout(width='200px')), ticker_select], layout=row_layout))

    result = Box(rows, layout=Layout(**col_layout))
    display(result)
    
    run_eval_button = Button(description="Run_Evaluation", button_style='success', margin=leftm(650))
    display(run_eval_button)
    run_eval_button.on_click(run_single_factor)


In [21]:
individual_eval_button = Button(description="Individual_Evaluation", button_style='success', margin=leftm(650))
display(individual_eval_button)
individual_eval_button.on_click(alpha_single)

Button(button_style='success', description='Individual_Evaluation', style=ButtonStyle())